In [3]:
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc, dash_table
import sqlite3
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import scipy.stats as stats
import plotly.express as px
import re
import random
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold
from scipy.spatial.distance import pdist, squareform
import umap

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

_conn = sqlite3.connect("airbnb_cartagena.sqlite")
df_attr = pd.read_sql_query("SELECT * FROM Attributes", _conn, dtype={"ID": str})
df_ts = pd.read_sql_query("SELECT * FROM TimeSeriesRaw", _conn, dtype={"ID": str})
df_ts_interp = pd.read_sql_query("SELECT * FROM TimeSeriesInterpolated", _conn, dtype={"ID": str})
_conn.close()

red = "#7e0d24"  # dark red color for plots
dates = [col for col in df_ts.columns if re.fullmatch(r"\d{1,2}/\d{1,2}/\d{4}", col)]
df_ts_interp = df_ts_interp.dropna(subset=dates, how="any").reset_index(drop=True)

In [5]:
def build_umap_and_distances():
    # Drop some columns that are not needed
    df_temp = df_attr.copy()
    """
    df_temp=df_temp[[
    'Name', 'Host', 'Base fee', 'Cleaning fee', 'URL', 'ID', 'latitude',
    'longitude', 'Property type', 'Person capacity', 'accuracy_rating',
    'checking_rating', 'cleanliness_rating', 'communication_rating',
    'location_rating', 'value_rating', 'satisfaction_rating', 'Reviews',
    'Bedrooms', 'Beds', 'Baths', 'City skyline view', 'Beach view',
    'Sea/Lake view', 'Hot water', 'Jacuzzi', 'Shared pool', 'Shared gym',
    'Patio or balcony', 'Outdoor furniture', 'Outdoor playground',
    'Elevator', 'Carport', 'Dedicated workspace', 'AC', 'Heating', 'TV',
    'Cable TV', 'Wifi', 'Laundry service', 'Kitchen', 'Dining table',
    'Microwave', 'Dishes and silverware', 'Refrigerator', 'Stove', 'Keypad',
    'Washer', 'Pets allowed', 'Crib', 'Security cameras', 'Lock on door']]
    to_keep = [
        "Keypad", "Lock on door", "Smoke detector", "Security cameras", "AC", "Heating", 
        "Patio or balcony", "Stove", "Elevator", "Refrigerator", "Kitchen", "Wifi", 
        "TV", "Jacuzzi", "Carport", "Hot water", 
    ]
    df_temp = df_temp[["ID", "Base fee"] + to_keep]
    """
    df_temp = df_temp.iloc[:, :21]

    # Melt time series data to long format
    df_prices = (
        df_ts_interp.copy()
        .melt(id_vars="ID", value_vars=dates, var_name="Date", value_name="Value")
        .assign(Date=lambda d: pd.to_datetime(d["Date"], dayfirst=True))
    )

    # Summarize per‐listing log‐price mean, std, and trend
    def summarize(group):
        #y = np.log1p(group["Value"].replace(0, np.nan))  # Avoid log(0)
        y = np.log1p(group["Value"])
        #y = group["Value"]
        days = (group["Date"] - group["Date"].min()).dt.days.values.reshape(-1,1)
        lr = LinearRegression().fit(days, y) if len(np.unique(days))>1 else None
        return pd.Series({
            "price_mean": y.mean(),
            "price_std":  y.std(),
            "price_trend": lr.coef_[0] if lr else 0.0
        })
    df_price_summary = (
        df_prices
        .groupby("ID", group_keys=False)
        .apply(summarize)
    )
    df_merged = df_attr.merge(df_price_summary, on="ID")

    # Filter out near‐constant / low‐variance features
    selector = VarianceThreshold(threshold=0.1)
    X = selector.fit_transform(df_merged.select_dtypes("number"))
    to_keep = df_merged.select_dtypes("number").columns[selector.get_support()]

    # Drop highly correlated (>0.9)
    df_reduced = pd.DataFrame(X, columns=to_keep)
    corr = df_reduced.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape, dtype=bool), k=1))
    to_drop = [c for c in upper.columns if (upper[c] > 0.9).any()]
    df_space = df_reduced.drop(columns=to_drop)

    # Scale + UMAP embedding
    X_scaled = StandardScaler().fit_transform(df_space)
    umap_proj = umap.UMAP(n_components=3, n_neighbors=30, min_dist=0.1, random_state=69).fit_transform(X_scaled)

    # Build distance matrix and UMAP space DataFrame
    df_space = df_merged.loc[df_space.index, ['ID','Base fee']].reset_index(drop=True)
    df_space[['UMAP1','UMAP2','UMAP3']] = umap_proj
    dist_matrix = squareform(pdist(df_space.drop(columns=["ID", 'Base fee']).values, metric="euclidean"))
    df_dist = pd.DataFrame(dist_matrix, index=df_space['ID'], columns=df_space['ID'])
    
    return df_space, df_dist, dist_matrix, df_prices

In [7]:
df_space, df_dist, dist_matrix, df_prices = build_umap_and_distances()

# Model 1

In [ ]:
import sqlite3, re
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import umap, gudhi as gd
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

# 1) LOAD DATA FROM SQLITE
_conn = sqlite3.connect("airbnb_cartagena.sqlite")
df_attr      = pd.read_sql("SELECT * FROM Attributes", _conn, dtype={"ID": str})
df_ts_interp = pd.read_sql("SELECT * FROM TimeSeriesInterpolated", _conn, dtype={"ID": str})
_conn.close()
dates = [c for c in df_ts_interp.columns if re.fullmatch(r"\d{1,2}/\d{1,2}/\d{4}", c)]
df_ts_interp = df_ts_interp.dropna(subset=dates).reset_index(drop=True)
df_attr = df_attr.set_index("ID").reindex(df_ts_interp["ID"].values).copy() # reindex attributes to match the time-series rows

# 2) PRICE‐TIER TARGET
df_attr["price_tier"] = pd.qcut(df_attr["Base fee"], 3, labels=["Low","Mid","High"])

# 3) TIME‐SERIES SUMMARY FEATURES
def make_ts(df, dates):
    rows = []
    for vals in df[dates].values.astype(float):
        y = np.log1p(vals)
        d = np.arange(len(y))
        coef = np.polyfit(d, y, 1)[0] if len(y)>1 else 0
        s = pd.Series(y)
        row = {
            "ts_mean":  y.mean(),
            "ts_std":   y.std(ddof=0),
            "ts_trend": coef,
            "ts_spike": (s.diff().abs()>0.1).sum() # type: ignore
        }
        for w in (7,14):
            r = s.rolling(w, min_periods=1).agg(["mean","std"]).iloc[-1]
            row[f"roll{w}_mean"], row[f"roll{w}_std"] = r["mean"], r["std"]
        rows.append(row)
    return pd.DataFrame(rows, index=df["ID"])
df_ts_feats = make_ts(df_ts_interp, dates)

# 4) UMAP “neighborhood” embedding on static + TS features
static_num  = df_attr.select_dtypes("number")
static_bool = df_attr.select_dtypes("bool").astype(int)
X_umap       = pd.concat([static_num, static_bool, df_ts_feats], axis=1).fillna(0)
proj         = umap.UMAP(n_components=2, random_state=42).fit_transform(X_umap)
df_attr["UMAP1"], df_attr["UMAP2"] = proj[:,0], proj[:,1] # type: ignore

# 5) Global TDA summaries on UMAP-distance matrix
umap_coords = df_attr[["UMAP1","UMAP2"]].values
dist_matrix = squareform(pdist(umap_coords, metric="euclidean"))
rips = gd.RipsComplex(distance_matrix=dist_matrix, # type: ignore
                             max_edge_length=dist_matrix.max()) 
st = rips.create_simplex_tree(max_dimension=2)
st.compute_persistence()
pers = st.persistence()

tda_feats = {"H0_sum_life":0.0, "H1_sum_life":0.0}
for dim,(b,d) in pers:
    life = (d if d!=float("inf") else dist_matrix.max()) - b
    if dim==0: tda_feats["H0_sum_life"] += life
    if dim==1: tda_feats["H1_sum_life"] += life

# broadcast across all listings
df_tda_feats = pd.DataFrame([tda_feats]*len(df_attr),
                            index=df_attr.index)

# 6) Assemble everything
df = df_attr.join(df_ts_feats).join(df_tda_feats)
num_cols = df.select_dtypes("number").columns
df[num_cols] = df[num_cols].fillna(0)
y        = df["price_tier"]
X_static = df[["Cleaning fee","Bedrooms","Baths","UMAP1","UMAP2"]]
X_ts     = df_ts_feats
X_tda    = df_tda_feats
X_amen   = df.select_dtypes("bool").astype(int)

# 7) Pipeline + model
pre = ColumnTransformer([
    ("num",  Pipeline([("impute",SimpleImputer()),("scale",StandardScaler())]),
     list(X_static)+list(X_ts)+list(X_tda)),
    ("amen", OneHotEncoder(drop="first"), X_amen.columns.tolist())
])
model = Pipeline([
    ("prep", pre),
    ("clf",  XGBClassifier(
        n_estimators=200, max_depth=4, learning_rate=0.05,
        use_label_encoder=False, eval_metric="mlogloss", random_state=0
    ))
])
le    = LabelEncoder()
y_enc = le.fit_transform(y)

# 8) 5‐fold CV
cv     = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
X_all  = pd.concat([X_static, X_ts, X_tda, X_amen], axis=1)
scores = cross_val_score(model, X_all, y_enc, cv=cv, scoring="accuracy")
print("5-fold CV accuracy:", np.round(scores,3))
print("Mean CV accuracy:", np.round(scores.mean(),3))

# 8) hyperparameter tuning with 5-fold CV
param_grid = {
    "clf__n_estimators":    [100, 200, 300],
    "clf__max_depth":       [3, 4, 6],
    "clf__learning_rate":   [0.01, 0.05, 0.1],
    "clf__subsample":       [0.7, 0.9, 1.0],
    "clf__colsample_bytree":[0.7, 0.9, 1.0],
}
grid = GridSearchCV(
    model,
    param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=69),
    scoring="accuracy",
    n_jobs=-1,
    verbose=1,
)

grid.fit(X_all, y_enc)

print("Best CV score:", round(grid.best_score_, 3))
print("Best params:    ", grid.best_params_)

# 9) Evaluate tuned model on hold-out split
X_tr, X_te, y_tr, y_te = train_test_split(
    X_all, y_enc, stratify=y_enc, test_size=0.25, random_state=69
)

best_model = grid.best_estimator_
best_model.fit(X_tr, y_tr)
y_pr = best_model.predict(X_te)

print("Hold-out accuracy:", round(accuracy_score(y_te, y_pr), 3))
print(classification_report(
    le.inverse_transform(y_te),
    le.inverse_transform(y_pr),
    target_names=le.classes_
))


5-fold CV accuracy: [0.75  0.625 0.625 0.938 0.533]
Mean CV accuracy: 0.694
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best CV score: 0.812
Best params:     {'clf__colsample_bytree': 0.7, 'clf__learning_rate': 0.05, 'clf__max_depth': 3, 'clf__n_estimators': 100, 'clf__subsample': 0.9}
Hold-out accuracy: 0.95
              precision    recall  f1-score   support

        High       0.88      1.00      0.93         7
         Low       1.00      1.00      1.00         7
         Mid       1.00      0.83      0.91         6

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.96      0.95      0.95        20



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    average_precision_score
)

# 10) Save the model and metrics
y_pred = best_model.predict(X_te)
y_score = best_model.predict_proba(X_te)
report = classification_report(
    le.inverse_transform(y_te),
    le.inverse_transform(y_pred),
    target_names=le.classes_,
    output_dict=True
)
metrics_summary = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1-score"],
    "Value": [
        round(accuracy_score(y_te, y_pred), 3),
        round(report["weighted avg"]["precision"], 3), # type: ignore
        round(report["weighted avg"]["recall"], 3), # type: ignore
        round(report["weighted avg"]["f1-score"], 3) # type: ignore
    ]
}
metrics_df = pd.DataFrame(metrics_summary)
metrics_df.to_csv("models/model1/price_tier_metrics.csv", index=False)

# 11) Confusion matrix
cm = confusion_matrix(y_te, y_pred)
cm_df = pd.DataFrame(cm, index=le.classes_, columns=le.classes_)
cm_df.to_csv("models/model1/price_tier_confusion_matrix.csv")

# 12) Precision-Recall curves
pr_colors = ["#d79c9c", red, "#c71a37"]
pr_traces = []
for i, cls in enumerate(le.classes_):
    p, r, _ = precision_recall_curve((y_te==i).astype(int), y_score[:,i])
    ap = average_precision_score((y_te==i).astype(int), y_score[:,i])
    pr_traces.append(
        go.Scatter(
            x=r,
            y=p,
            mode="lines",
            name=f"{cls} (AP={ap:.2f})",
            line=dict(color=pr_colors[i % len(pr_colors)])
        )
    )
pr_layout = go.Layout(
    title="Precision–Recall Curves by Class",
    template="plotly_dark",
    xaxis_title="Recall",
    yaxis_title="Precision",
    legend=dict(bgcolor="rgba(0,0,0,0)")
)
pr_fig = go.Figure(data=pr_traces, layout=pr_layout)
pr_fig.write_json("models/model1/price_tier_pr_curves.json")


# Model 3

In [4]:
import sqlite3, re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import umap, gudhi as gd
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV

# 1) LOAD DATA FROM SQLITE
_conn = sqlite3.connect("airbnb_cartagena.sqlite")
df_attr       = pd.read_sql("SELECT * FROM Attributes", _conn, dtype={"ID": str})
df_ts_interp  = pd.read_sql("SELECT * FROM TimeSeriesInterpolated", _conn, dtype={"ID": str})
_conn.close()
dates = [c for c in df_ts_interp.columns if re.fullmatch(r"\d{1,2}/\d{1,2}/\d{4}", c)]
df_ts_interp = df_ts_interp.dropna(subset=dates).reset_index(drop=True)
df_attr      = df_attr.set_index("ID").reindex(df_ts_interp["ID"]).reset_index()

# 2) TARGET: average of all *_rating ≥4.8
rating_cols = [c for c in df_attr.columns if c.endswith("_rating")]
df_attr["is_superhost"] = (df_attr[rating_cols].mean(axis=1) >= 4.8).astype(int)

# 3) TIME‐SERIES SUMMARY FEATURES
def make_ts(df, dates):
    rows = []
    for vals in df[dates].values.astype(float):
        y = np.log1p(vals)
        d = np.arange(len(y))
        coef = np.polyfit(d, y, 1)[0] if len(y)>1 else 0
        s = pd.Series(y)
        r7  = s.rolling(7, 1).agg(["mean","std"]).iloc[-1]
        r14 = s.rolling(14,1).agg(["mean","std"]).iloc[-1]
        rows.append({
            "ts_mean": y.mean(), "ts_std": y.std(ddof=0), "ts_trend": coef,
            "roll7_mean": r7["mean"], "roll7_std": r7["std"],
            "roll14_mean": r14["mean"], "roll14_std": r14["std"],
            "ts_spikes": (s.diff().abs()>0.1).sum()
        })
    return pd.DataFrame(rows, index=df["ID"])
df_ts_feats = make_ts(df_ts_interp, dates)
df_ts_feats.index = df_attr.index

# 4) UMAP on static + ts
static_num  = df_attr.select_dtypes(include="number").drop(columns=["is_superhost"])
static_bool = df_attr.select_dtypes(include="bool").astype(int)
X_umap      = pd.concat([static_num, static_bool, df_ts_feats], axis=1).fillna(0)
proj        = umap.UMAP(n_components=2, random_state=69).fit_transform(X_umap)
df_attr["UMAP1"], df_attr["UMAP2"] = proj[:,0], proj[:,1] # type: ignore

# 5) TDA on price volatility window=14
def make_tda(df, dates, w=14):
    rows=[]
    for vals in df[dates].values.astype(float):
        N = len(vals)-w+1
        cloud = np.stack([vals[i:i+w] for i in range(N)])
        st = gd.RipsComplex(points=cloud, max_edge_length=vals.ptp()) \
               .create_simplex_tree(max_dimension=2)
        st.compute_persistence()
        h0,h1 = [],[]
        for dim,(b,e) in st.persistence():
            life = ((e if e != np.inf else vals.ptp()) - b)
            (h0 if dim==0 else h1).append(life)
        rows.append({
            "H0_max": max(h0, default=0), "H0_sum": sum(h0),
            "H1_max": max(h1, default=0), "H1_sum": sum(h1), "H1_cnt": len(h1)
        })
    return pd.DataFrame(rows, index=df["ID"])

df_tda_feats = make_tda(df_ts_interp, dates)

# 6) ASSEMBLE
df = df_attr.set_index("ID") \
    .join(df_ts_feats).join(df_tda_feats) \
    .fillna(0)
y = df["is_superhost"]
X_num  = df.select_dtypes(include="number").drop(columns=["is_superhost"])
X_amen = df.select_dtypes(include="bool").astype(int)
X_all  = pd.concat([X_num, X_amen], axis=1)

# 7) SPLIT
X_tr, X_te, y_tr, y_te = train_test_split(
    X_all, y, stratify=y, test_size=0.5, random_state=69)

# 8) PREPROCESSOR
pre = ColumnTransformer([
    ("num", Pipeline([("impute",SimpleImputer()),("scale",StandardScaler())]),
     X_num.columns),
    ("amen", OneHotEncoder(drop="first"), X_amen.columns)
])

# 9) Define ensemble
ensemble = VotingClassifier(
    estimators=[
        ("log", Pipeline([("prep", pre),
                          ("clf", LogisticRegression(penalty="l1", solver="saga",
                                                     max_iter=2000))])),
        ("rf",  Pipeline([("prep", pre),
                          ("clf", RandomForestClassifier(random_state=69))]))
    ],
    voting="soft"
)

# 10) Hyperparameter tunning with 5-fold CV
param_grid = {
    "log__clf__C": [0.01, 0.1, 1, 10],
    "rf__clf__n_estimators": [100, 200, 500],
    "rf__clf__max_depth": [None, 5, 10],
}
grid = GridSearchCV(
    ensemble,
    param_grid,
    cv=StratifiedKFold(5, shuffle=True, random_state=69),
    scoring="accuracy",
    n_jobs=-1,
    verbose=1
)
grid.fit(X_all, y)
print("Best CV score:", round(grid.best_score_, 3))
print("Best params:   ", grid.best_params_)

# 12) Evaluate on hold-out
best = grid.best_estimator_
best.fit(X_tr, y_tr)
y_pr = best.predict(X_te)
print("\n=== Tuned Ensemble ===")
print("Test accuracy :", round(accuracy_score(y_te, y_pr), 3))
print(classification_report(y_te, y_pr, digits=3))
print("Confusion Matrix:\n", confusion_matrix(y_te, y_pr))


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best CV score: 0.962
Best params:    {'log__clf__C': 1, 'rf__clf__max_depth': None, 'rf__clf__n_estimators': 100}

=== Tuned Ensemble ===
Test accuracy : 0.95
              precision    recall  f1-score   support

           0      0.909     0.909     0.909        11
           1      0.966     0.966     0.966        29

    accuracy                          0.950        40
   macro avg      0.937     0.937     0.937        40
weighted avg      0.950     0.950     0.950        40

Confusion Matrix:
 [[10  1]
 [ 1 28]]


In [27]:
import pandas as pd
import plotly.express as px
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, auc
import plotly.graph_objs as go

# 13) Save the model and metrics
y_pred = best.predict(X_te)
acc    = accuracy_score(y_te, y_pred)
class_names = ["Not Superhost", "Superhost"]
report = classification_report(
    y_te,
    y_pred,
    target_names=class_names,
    output_dict=True
)
metrics_summary = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1-score"],
    "Value": [
        round(acc, 3),
        round(report["weighted avg"]["precision"], 3), # type: ignore
        round(report["weighted avg"]["recall"],    3), # type: ignore
        round(report["weighted avg"]["f1-score"],  3), # type: ignore
    ]
}
metrics_df = pd.DataFrame(metrics_summary)
metrics_df.to_csv("models/model2/metrics_summary.csv", index=False)

# 14) Confusion matrix
cm    = confusion_matrix(y_te, y_pred)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
cm_df.to_csv("models/model2/confusion_matrix.csv")

# 15) Feature importances (from RF inside the ensemble)
rf_pipeline = best.named_estimators_['rf']
importances = rf_pipeline.named_steps['clf'].feature_importances_
num_feats  = list(X_num.columns)
amen_feats = list(X_amen.columns)
all_feats  = num_feats + amen_feats
fi_df = (
    pd.DataFrame({"feature": all_feats, "importance": importances})
      .sort_values("importance", ascending=False)
)
#fi_df.to_csv("models/model2/feature_importances.csv", index=False)
# 16) Plot top 20 feature importances
top_n = fi_df.head(20)
fig   = px.bar(
    top_n,
    x="importance",
    y="feature",
    orientation="h",
    title="Top 20 Feature Importances (RF)",
    labels={"importance":"Importance", "feature":""},
    template="plotly_dark",
    color_discrete_sequence=[red]
)
fig.update_layout(yaxis=dict(autorange="reversed"), height=600)
fig.write_json("models/model2/feature_importances.json")

# 17) ROC curve for Superhost classification
y_score = best.predict_proba(X_te)[:, 1]   # probability of class “1”
y_true  = y_te                            # 0/1 labels
fpr, tpr, _ = roc_curve(y_true, y_score)
roc_auc     = auc(fpr, tpr)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=fpr,
    y=tpr,
    mode="lines",
    name=f"Superhost (AUC={roc_auc:.2f})",
    line=dict(color=red, width=3),
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=[0, 1],
    y=[0, 1],
    mode="lines",
    line=dict(color="white", width=1, dash="dash"),
    showlegend=False
))
fig.update_layout(
    title=f"ROC Curve (AUC = {roc_auc:.2f})",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    template="plotly_dark",
    legend=dict(bgcolor="rgba(0,0,0,0)"),
    height=400
)
fig.write_json("models/model2/roc_curve.json")

In [28]:
import plotly.io as pio

def model2_roc_curves():
    """
    Load and return the ROC curve figure for model 2.
    """
    fig = pio.read_json("models/model2/roc_curve.json")
    fig.update_layout(height=400)
    return fig

model2_roc_curves()

In [ ]:
import plotly.io as pio

def model2_feature_importances():
    """
    Load the pre-computed feature-importances JSON and return
    a dark-themed Plotly bar chart (top 20).
    """
    # read the JSON you wrote out
    fig = pio.read_json("models/model2/feature_importances.json")
    
    # enforce our styling
    fig.update_layout(
        title="Top 20 Feature Importances (Random Forest)",
        template="plotly_dark",
        xaxis_title="Importance",
        yaxis_title="",
        margin=dict(l=200, r=40, t=80, b=40),
        height=600,
    )
    # make sure the largest importance is at top
    fig.update_yaxes(autorange="reversed")
    
    return fig
model2_feature_importances()

# Model 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import lightgbm as lgb
import gudhi as gd

def build_occupancy_dataset(
    df_attr, df_ts, df_ts_interp, dates,
    roll_window=7,
    tda_edge=None
):
    """
    Returns X (features) and y (sold_out flag) for each (ID, date) 
    excluding the first roll_window days.
    """
    # 1) Build long tables
    # a) raw availability flag
    df_avail = (
        df_ts
        .melt(id_vars="ID", value_vars=dates,
              var_name="Date", value_name="Price")
        .assign(Date=lambda d: pd.to_datetime(d["Date"], dayfirst=True))
    )
    df_avail["sold_out"] = df_avail["Price"].isna().astype(int)
    
    # b) interpolated price for stats & TDA
    df_int = (
        df_ts_interp
        .melt(id_vars="ID", value_vars=dates,
              var_name="Date", value_name="InterpPrice")
        .assign(Date=lambda d: pd.to_datetime(d["Date"], dayfirst=True))
    )

    # 2) Rolling features per listing
    feats = []
    for lid, group in df_avail.groupby("ID"):
        g = group.sort_values("Date").set_index("Date")
        # rolling availability rate
        g["roll_avail_rate"] = 1 - g["sold_out"].rolling(roll_window).mean()
        # merge price stats
        pi = df_int[df_int["ID"]==lid].sort_values("Date").set_index("Date")
        g = g.join(pi["InterpPrice"])
        g["roll_price_mean"] = g["InterpPrice"].rolling(roll_window).mean()
        g["roll_price_std"]  = g["InterpPrice"].rolling(roll_window).std()
        # TDA summary on the last window
        # build the cloud once per date
        if tda_edge is None:
            tda_edge = g["InterpPrice"].max() - g["InterpPrice"].min()
        h0, h1 = [], []
        prices = g["InterpPrice"].values
        for i in range(len(prices)):
            if i < roll_window-1:
                h0.append(np.nan); h1.append(np.nan)
            else:
                window = prices[i-roll_window+1:i+1]
                cloud = np.stack([window[j:j+roll_window] 
                                  for j in range(len(window)-roll_window+1+roll_window)])
                # simplify: use the 1D window itself as cloud of shape (roll_window, roll_window)
                cloud = np.stack([window]*roll_window).T
                st = gd.RipsComplex(points=cloud, max_edge_length=tda_edge) \
                        .create_simplex_tree(max_dimension=2)
                st.compute_persistence()
                s0=s1=0.0
                for dim,(b,d) in st.persistence():
                    life = (d if d!=float("inf") else tda_edge) - b
                    if dim==0: s0+=life
                    if dim==1: s1+=life
                h0.append(s0); h1.append(s1)
        g["tda_H0_sum"] = h0
        g["tda_H1_sum"] = h1
        
        g["ID"] = lid
        feats.append(g.reset_index())
    df_feat = pd.concat(feats, ignore_index=True)
    
    # 3) Seasonality
    df_feat["dow"]   = df_feat["Date"].dt.weekday
    df_feat["month"] = df_feat["Date"].dt.month
    
    # 4) Static attributes + amenity one-hots
    dfA = df_attr.set_index("ID")
    static = dfA[["Cleaning fee","Bedrooms","Baths"]]
    amenities = dfA.select_dtypes(bool).astype(int)
    ohe = OneHotEncoder(drop="first", sparse=False)
    X_amen = pd.DataFrame(
        ohe.fit_transform(amenities),
        index=amenities.index,
        columns=ohe.get_feature_names_out(amenities.columns)
    )
    static = pd.concat([static, X_amen], axis=1)
    df_feat = df_feat.join(static, on="ID")

    # 5) Drop rows with any NaNs (warm-up window)
    df_feat = df_feat.dropna(subset=[
        "roll_avail_rate","roll_price_mean","roll_price_std",
        "tda_H0_sum","tda_H1_sum"
    ])

    # 6) Assemble X, y
    y = df_feat["sold_out"]
    X = df_feat[[
        "roll_avail_rate","roll_price_mean","roll_price_std",
        "tda_H0_sum","tda_H1_sum",
        "dow","month"
    ] + static.columns.tolist()]
    return X, y

# --- use it ---
X, y = build_occupancy_dataset(df_attr, df_ts, df_ts_interp, dates)

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=0
)

# scale numeric columns
num_cols = [
    "roll_avail_rate","roll_price_mean","roll_price_std",
    "tda_H0_sum","tda_H1_sum"
]
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

# --- train LightGBM ---
clf = lgb.LGBMClassifier(n_estimators=200, random_state=0)
clf.fit(X_train, y_train)

# --- evaluate ---
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


# Model 3

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import NearestNeighbors
import gudhi as gd

# 1) Prepare the joined DataFrame
# --------------------------------
# assume df_attr, df_space are already loaded (from build_umap_and_distances)
# df_attr: contains ID, static features, boolean amenity columns, and either "superhost" or rating
# df_space: contains ID, UMAP1, UMAP2, UMAP3

# Merge on ID
df = df_attr.merge(df_space[['ID','UMAP1','UMAP2','UMAP3']], on='ID')

# Define target
if 'superhost' in df.columns:
    df['target'] = df['superhost'].astype(int)
else:
    df['target'] = (df['satisfaction_rating'] >= 4.8).astype(int)

# 2) Build TDA summarizer over each listing's UMAP neighborhood
# -------------------------------------------------------------
def compute_local_tda(df_space, k=10, max_dim=1):
    """
    For each listing in df_space, find its k nearest neighbors in UMAP space,
    build a Rips complex on that small cloud, and return sum of lifetimes
    in H0 and H1 as features.
    """
    coords = df_space[['UMAP1','UMAP2','UMAP3']].values
    nb = NearestNeighbors(n_neighbors=k, metric='euclidean').fit(coords)
    dists, idxs = nb.kneighbors(coords)
    
    tda_feats = []
    for i, neighbors in enumerate(idxs):
        cloud = coords[neighbors]
        # set max_edge to the max pairwise distance in this cloud
        max_edge = np.max(pdist(cloud))
        rips = gd.RipsComplex(points=cloud, max_edge_length=max_edge)
        st = rips.create_simplex_tree(max_dimension=max_dim+1)
        st.compute_persistence()
        sum_life = {d:0.0 for d in range(max_dim+1)}
        for dim, (b,d) in st.persistence():
            life = (d if d!=float('inf') else max_edge) - b
            if dim<=max_dim:
                sum_life[dim] += life
        tda_feats.append((sum_life[0], sum_life.get(1,0.0)))
    
    df_tda = pd.DataFrame(tda_feats, columns=['tda_H0_sum','tda_H1_sum'], index=df_space.index)
    return df_tda

from scipy.spatial.distance import pdist

# compute and join
df_tda = compute_local_tda(df_space, k=10, max_dim=1)
df = df.join(df_tda, on=df.index)

# 3) Build feature matrix
# ------------------------
# numeric static features
num_feats = [
    'cleaning_fee','Bedrooms','Beds','Baths',
    'accuracy_rating','cleanliness_rating','communication_rating',
    'location_rating','value_rating','satisfaction_rating'
]
# amenities (bool → int)
amen_cols = df_attr.select_dtypes(include='bool').columns.tolist()

# pipeline: scale numeric, one-hot amenities, pass through UMAP coords & TDA feats
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_feats),
    ('amen', OneHotEncoder(drop='first'), amen_cols),
    ('umap', 'passthrough', ['UMAP1','UMAP2','UMAP3']),
    ('tda', 'passthrough', ['tda_H0_sum','tda_H1_sum'])
])

X = df[num_feats + amen_cols + ['UMAP1','UMAP2','UMAP3','tda_H0_sum','tda_H1_sum']]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=0
)

# 4) Fit and evaluate models
# ---------------------------

models = {
    "Logistic L1": LogisticRegression(penalty='l1', solver='saga', max_iter=2000, random_state=0),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=0),
    "Linear SVM": LinearSVC(max_iter=2000, random_state=0)
}

for name, clf in models.items():
    pipe = Pipeline([
        ('pre', preprocessor),
        ('clf', clf)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.3f}")
    print(classification_report(y_test, y_pred, digits=3))


TypeError: a bytes-like object is required, not 'RangeIndex'

# Model 4

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import gudhi as gd

# --- 1) Prepare target via volatility features ------------------------------

# 4.3.0 from above
def compute_volatility_features(df_ts_interp, dates, spike_thresh=0.1):
    mat = df_ts_interp.set_index("ID")[dates].astype(float)
    mean = mat.mean(axis=1)
    dev = mat.sub(mean, axis=0).div(mean, axis=0)
    feats = pd.DataFrame({
        'std_dev':    dev.std(axis=1),
        'max_dev':    dev.abs().max(axis=1),
        'spike_freq': (dev.abs()>spike_thresh).sum(axis=1)/dev.shape[1]
    })
    feats.index.name = 'ID'
    return feats.round(3)

feats = compute_volatility_features(df_ts_interp, dates)

# Binarize around median
median_std = feats['std_dev'].median()
feats['target'] = (feats['std_dev'] > median_std).astype(int)  # 1 = volatile, 0 = stable

# --- 2) Build periodicity feature: count of H1 loops in 7-day window -------

def count_h1_loops(listing_id, window=7):
    row = df_ts_interp.loc[df_ts_interp['ID']==listing_id, dates].values.flatten().astype(float)
    N = len(row) - window + 1
    if N <= 0: 
        return 0
    # stack full sliding‐window cloud
    cloud = np.stack([row[i:i+window] for i in range(N)])
    rips = gd.RipsComplex(points=cloud, max_edge_length=np.ptp(row))
    st = rips.create_simplex_tree(max_dimension=2)
    st.compute_persistence()
    # count H1 intervals
    return sum(1 for dim,(b,d) in st.persistence() if dim==1 and d> b)

feats['h1_count'] = [count_h1_loops(lid) for lid in feats.index]

# --- 3) Assemble modeling DataFrame -----------------------------------------

# static + time‐series: pull from df_attr and price‐summary
df_price_summary = build_umap_and_distances()[-1] \
    .groupby("ID").apply(lambda g: pd.Series({
        'price_mean': np.log1p(g.Value).mean(),
        'price_std':  np.log1p(g.Value).std(),
        'price_trend': LinearRegression().fit(
            ((g.Date-g.Date.min()).dt.days.values.reshape(-1,1)),
            np.log1p(g.Value)
        ).coef_[0]
    }))
df_base = df_attr.set_index('ID').join(df_price_summary).join(feats)

# features lists
num_feats = [
    'cleaning_fee','Bedrooms','Beds','Baths',
    'price_mean','price_std','price_trend',
    'std_dev','max_dev','spike_freq','h1_count'
]
amen = df_attr.select_dtypes('bool').columns.tolist()

X = df_base[num_feats + amen]
y = df_base['target']

# --- 4) Preprocessing + model pipeline -------------------------------------

pre = ColumnTransformer([
    ('num', StandardScaler(), num_feats),
    ('amen', OneHotEncoder(drop='first'), amen)
])

models = {
    'RF': RandomForestClassifier(n_estimators=200, random_state=0),
    'XGB': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=0)
}

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=0
)

for name, clf in models.items():
    pipe = Pipeline([('pre', pre), ('clf', clf)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))


# Model 5

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist
import tensorflow as tf
from tensorflow.keras import layers, models

# 1) Build our dataset ------------------------------------------------------

WINDOW = 30

# 1a) Extract last-30-day windows (normalized) + linear trend target
records = []
for idx, row in df_ts_interp.set_index("ID").iterrows():
    prices = row[dates].astype(float).values
    if len(prices) < WINDOW: 
        continue
    window = prices[-WINDOW:]
    # normalize
    window_norm = (window - window.mean()) / (window.std() + 1e-6)
    # compute trend slope
    x = np.arange(WINDOW).reshape(-1,1)
    slope = np.polyfit(x.flatten(), window, 1)[0]
    # discretize target
    if slope > 0.01:
        label = 2  # up
    elif slope < -0.01:
        label = 0  # down
    else:
        label = 1  # flat
    records.append({
        'ID': idx,
        'window': window_norm,
        'trend': label
    })

df_ws = pd.DataFrame(records)
X_series = np.stack(df_ws['window'].values)        # shape (n_samples, 30)
y = df_ws['trend'].values                          # 0=down,1=flat,2=up

# train/test split
X_train_s, X_test_s, y_train, y_test = train_test_split(
    X_series, y, test_size=0.3, stratify=y, random_state=42
)

# 1b) Feature‐extraction for RF & k-NN: persistence summaries
def extract_tda_features(windows, max_dim=1):
    feats = []
    for w in windows:
        cloud = np.stack([w[i:i+max_dim+1] for i in range(len(w)-max_dim)])  # tiny embedding
        rips = gd.RipsComplex(points=cloud, max_edge_length=np.ptp(w))
        st = rips.create_simplex_tree(max_dimension=max_dim+1)
        st.compute_persistence()
        pairs = st.persistence()
        # sum of lifetimes per dim
        sums = {d: sum((d1-d0) for dd,(d0,d1) in pairs if dd==d and d1<np.inf) for d in range(max_dim+1)}
        # count intervals
        counts = {f'c{d}': sum(1 for dd,(_,_d1) in pairs if dd==d) for d in range(max_dim+1)}
        feats.append([sums.get(d,0) for d in range(max_dim+1)] +
                     [counts[f'c{d}'] for d in range(max_dim+1)])
    cols = [f'sum_life_H{d}' for d in range(max_dim+1)] + [f'count_H{d}' for d in range(max_dim+1)]
    return pd.DataFrame(feats, columns=cols)

df_train_tda = extract_tda_features(X_train_s, max_dim=1)
df_test_tda  = extract_tda_features(X_test_s,  max_dim=1)

# --- 2) Random Forest on TDA features --------------------------------------

rf_pipe = Pipeline([
    ('scale', StandardScaler()),
    ('rf', RandomForestClassifier(n_estimators=200, random_state=0))
])
rf_pipe.fit(df_train_tda, y_train)
y_pred_rf = rf_pipe.predict(df_test_tda)
print("=== Random Forest on TDA features ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, digits=3))

# --- 3) DTW + k-NN using raw windowed series -------------------------------

# Precompute pairwise DTW distances between test and train
# (for simplicity, here we approximate DTW by Euclidean on windows;
# for real DTW use dtaidistance or fastdtw)
dist_mat = cdist(X_test_s, X_train_s, metric='euclidean')
knn = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
knn.fit(dist_mat, y_train)  # note: sklearn expects fit(X, y) but with precomputed, X is distance matrix
# To predict we need the distance from test to train again:
dist_test = dist_mat
y_pred_knn = knn.predict(dist_test)
print("=== k-NN (approx. DTW) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn, digits=3))

# --- 4) Simple 1D CNN on raw series -----------------------------------------

# reshape for CNN: (batch, time, 1)
X_train_c = X_train_s[..., np.newaxis]
X_test_c  = X_test_s[..., np.newaxis]

cnn = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(WINDOW,1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn.fit(X_train_c, y_train, validation_split=0.1, epochs=20, batch_size=32, verbose=2)

loss, acc = cnn.evaluate(X_test_c, y_test, verbose=0)
print("\n=== 1D CNN ===")
print(f"Test accuracy: {acc:.3f}")


# Model 6

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# --- 0) Prepare “ground-truth” segments via your unsupervised clusters -----

# am: DataFrame shape (n_listings, n_amenities) with 0/1
# labels: Series of cluster labels (strings) indexed by listing ID
am, labels = cluster_by_amenities(k=4)

# Build a single DataFrame of features + target
# Start with amenities one-hots:
df_seg = am.copy()
df_seg['segment'] = labels

# Add other static features: capacity, ratings, price_summary
# (we assume you already ran build_umap_and_distances or similar)
# here we recompute price_mean, price_std from df_prices for simplicity:
price_summary = (
    df_prices
    .groupby("ID")['Value']
    .agg(price_mean='mean', price_std='std')
)
# merge them in
df_seg = (
    df_seg
    .merge(price_summary, left_index=True, right_index=True)
    .merge(df_attr.set_index('ID')[['Person capacity']], left_index=True, right_index=True)
    .dropna()  # drop any listings missing fields
)

X = df_seg.drop(columns='segment').values
y = df_seg['segment'].astype(int).values  # convert to int for sklearn

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)

# --- 1) Multinomial Logistic Regression ------------------------------------

log_pipe = Pipeline([
    ('scale', StandardScaler()),
    ('logreg', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000))
])
log_pipe.fit(X_train, y_train)
y_pred_log = log_pipe.predict(X_test)

print("=== Multinomial Logistic Regression ===")
print(classification_report(y_test, y_pred_log, zero_division=0))

# --- 2) Gradient Boosting --------------------------------------------------

gb = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=0)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

print("=== Gradient Boosting ===")
print(classification_report(y_test, y_pred_gb, zero_division=0))

# --- 3) Interpretable Decision Tree ----------------------------------------

dt = DecisionTreeClassifier(max_depth=4, random_state=0)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print("=== Decision Tree (depth=4) ===")
print(classification_report(y_test, y_pred_dt, zero_division=0))

# Print the textual tree for interpretation:
feature_names = df_seg.drop(columns='segment').columns.tolist()
print("\nLearned Decision Tree rules:\n")
print(export_text(dt, feature_names=feature_names))

# --- 4) Confusion Matrices (optional) --------------------------------------

import seaborn as sns
import matplotlib.pyplot as plt

for name, y_pred in [('LogReg', y_pred_log), ('GB', y_pred_gb), ('DT', y_pred_dt)]:
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


# Model 7

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, hamming_loss
import tensorflow as tf
from tensorflow.keras import layers, models

# 1) BUILD FEATURE MATRIX X and MULTI-LABEL TARGET Y -----------------------

# static features from Attributes
static_feats = df_attr.set_index("ID")[
    ["Base fee", "Cleaning fee", "Bedrooms", "Beds", "Baths"] +
    [c for c in df_attr.columns if c.endswith("_rating")]
]

# volatility features
vol_feats = compute_volatility_features()

# UMAP coords
df_space, _, _, _ = build_umap_and_distances()
umap_feats = df_space.set_index("ID")[["UMAP1", "UMAP2", "UMAP3"]]

# price_time_series summaries
# (already computed in build_umap_and_distances as price_mean, price_std, price_trend)
# but if not, recompute quickly:
price_summary = (
    df_prices
    .groupby("ID")['Value']
    .agg(price_mean=lambda x: np.log1p(x).mean(),
         price_std = lambda x: np.log1p(x).std(),
         price_trend=lambda x: LinearRegression()
                              .fit(
                                  (pd.to_datetime(df_prices.loc[x.index,"Date"]) 
                                   - pd.to_datetime(df_prices.loc[x.index,"Date"]).min()
                                  ).dt.days.values.reshape(-1,1),
                                  np.log1p(x)
                              ).coef_[0]
        )
)

# combine all numeric features
X = (
    static_feats
    .join(vol_feats, how="inner")
    .join(umap_feats, how="inner")
    .join(price_summary, how="inner")
    .dropna()
)

# multi-label target: selected amenities one-hot
amenities = [
    "Wifi","Kitchen","Washer","Dryer","Air conditioning","Heating",
    "TV","Jacuzzi","Elevator","Patio or balcony"
]
Y = df_attr.set_index("ID")[amenities].loc[X.index].astype(int)

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)


# 2) MODEL A: MultiOutput Random Forest --------------------------------------

rf = MultiOutputClassifier(
    Pipeline([
        ('scale', StandardScaler()),
        ('rf', RandomForestClassifier(n_estimators=200, random_state=0))
    ])
)
rf.fit(X_train, y_train)
y_pred_rf = pd.DataFrame(rf.predict(X_test), index=y_test.index, columns=y_test.columns)

print("== Random Forest MultiOutput ==")
print("Hamming loss:", hamming_loss(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, zero_division=0))


# 3) MODEL B: One-vs-Rest Logistic Regression -------------------------------

ovr = OneVsRestClassifier(
    Pipeline([
        ('scale', StandardScaler()),
        ('lr', LogisticRegression(solver='liblinear'))
    ])
)
ovr.fit(X_train, y_train)
y_pred_ovr = pd.DataFrame(ovr.predict(X_test), index=y_test.index, columns=y_test.columns)

print("== One-vs-Rest Logistic ==")
print("Hamming loss:", hamming_loss(y_test, y_pred_ovr))
print(classification_report(y_test, y_pred_ovr, zero_division=0))


# 4) MODEL C: Deep MLP with Sigmoid Outputs ---------------------------------

# build a simple MLP
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

mlp = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(output_dim, activation='sigmoid')
])
mlp.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

# scale data
scaler = StandardScaler().fit(X_train)
Xtr = scaler.transform(X_train)
Xte = scaler.transform(X_test)

# train
mlp.fit(Xtr, y_train.values, 
        validation_split=0.1, epochs=20, batch_size=32, verbose=1)

# predict (threshold at 0.5)
y_prob = mlp.predict(Xte)
y_pred_mlp = (y_prob > 0.5).astype(int)
y_pred_mlp = pd.DataFrame(y_pred_mlp, index=y_test.index, columns=y_test.columns)

print("== Deep MLP ==")
print("Hamming loss:", hamming_loss(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp, zero_division=0))
